<a href="https://colab.research.google.com/github/MogoWhite/gesture_fake_video_detection/blob/main/MMAction2.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# MMAction2 Tutorial

- 用MMAction2的识别模型做一次推理
- 用新数据集训练一个新的识别模型
- 用MMAction2的时空检测模型做一次推理

In [ ]:
!python -V

Python 3.8.10


## 安装 MMAction2

In [ ]:
# 检查 nvcc，gcc 版本
!nvcc -V
!gcc --version

nvcc: NVIDIA (R) Cuda compiler driver
Copyright (c) 2005-2021 NVIDIA Corporation
Built on Sun_Feb_14_21:12:58_PST_2021
Cuda compilation tools, release 11.2, V11.2.152
Build cuda_11.2.r11.2/compiler.29618528_0
gcc (Ubuntu 9.4.0-1ubuntu1~20.04.1) 9.4.0
Copyright (C) 2019 Free Software Foundation, Inc.
This is free software; see the source for copying conditions.  There is NO
warranty; not even for MERCHANTABILITY or FITNESS FOR A PARTICULAR PURPOSE.



In [ ]:
# 安装 torch 及 torchvision
!pip install torch==1.9.0+cu111 torchvision==0.10.0+cu111 -f https://download.pytorch.org/whl/torch_stable.html

# 安装 mmcv-full
!pip install mmcv-full -f https://download.openmmlab.com/mmcv/dist/cu111/torch1.9.0/index.html

# 安装 mmaction2
!rm -rf mmaction2
!git clone https://github.com/open-mmlab/mmaction2.git
%cd mmaction2

!pip install -e .

# 安装其他可选依赖库
!pip install -r requirements/optional.txt

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
Looking in links: https://download.pytorch.org/whl/torch_stable.html
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╸ 2.0/2.0 GB 190.2 MB/s eta 0:00:01tcmalloc: large alloc 2041339904 bytes == 0x2de6000 @  0x7f0f5071e680 0x7f0f5073f824 0x5b3128 0x5bbc90 0x5f714c 0x64d800 0x527022 0x504866 0x56bbe1 0x569d8a 0x5f60c3 0x56bbe1 0x569d8a 0x5f60c3 0x56bbe1 0x569d8a 0x5f60c3 0x56bbe1 0x569d8a 0x5f60c3 0x56bbe1 0x569d8a 0x5f60c3 0x56bbe1 0x5f5ee6 0x56bbe1 0x569d8a 0x5f60c3 0x56cc92 0x569d8a 0x5f60c3
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╸ 2.0/2.0 GB 186.1 MB/s eta 0:00:01tcmalloc: large alloc 2551676928 bytes == 0x7c8ac000 @  0x7f0f5071e680 0x7f0f5073eda2 0x5f714c 0x64d800 0x527022 0x504866 0x56bbe1 0x569d8a 0x5f60c3 0x56bbe1 0x569d8a 0x5f60c3 0x56bbe1 0x569d8a 0x5f60c3 0x56bbe1 0x569d8a 0x5f60c3 0x56bbe1 0x569d8a 0x5f60c3 0x56bbe1 0x5f5ee6 0x56bbe1 0x569d8a 0x5f60c3 0x56cc92 0x569d8a 0x5f60c3

In [ ]:
# 克隆mmaction2项目
# %cd /content/
# !rm -rf mmaction2
# !git clone https://github.com/open-mmlab/mmaction2.git
!git clone https://github.com/wangruohui/mmaction2.git
%cd /content/mmaction2
!git checkout fix_nms_config

# 以可编辑的模式安装mmaction
!pip install -e .

# 安装一些额外的依赖
!pip install -r requirements/optional.txt

Cloning into 'mmaction2'...
remote: Enumerating objects: 11708, done.
remote: Total 11708 (delta 0), reused 0 (delta 0), pack-reused 11708
Receiving objects: 100% (11708/11708), 45.13 MiB | 46.50 MiB/s, done.
Resolving deltas: 100% (8162/8162), done.
/content/mmaction2
error: pathspec 'fix_nms_config' did not match any file(s) known to git
Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
Obtaining file:///content/mmaction2
  Preparing metadata (setup.py) ... done
  Attempting uninstall: mmaction2
    Found existing installation: mmaction2 0.24.1
    Can't uninstall 'mmaction2'. No files were found to uninstall.
  Running setup.py develop for mmaction2
Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
Ignoring dataclasses: markers 'python_version == "3.6"' don't match your environment


In [ ]:
# 检查torch的安装以及gpu的使用
import torch, torchvision
print(torch.__version__, torch.cuda.is_available())

# 检查MMAction2的安装
import mmaction
print(mmaction.__version__)

# 检查mmcv的安装
from mmcv.ops import get_compiling_cuda_version, get_compiler_version
print(get_compiling_cuda_version())
print(get_compiler_version())

1.9.0+cu111 True
0.24.1
11.1
GCC 7.3


/usr/local/lib/python3.8/dist-packages/mmcv/__init__.py:20: UserWarning: On January 1, 2023, MMCV will release v2.0.0, in which it will remove components related to the training process and add a data transformation module. In addition, it will rename the package names mmcv to mmcv-lite and mmcv-full to mmcv. See https://github.com/open-mmlab/mmcv/blob/master/docs/en/compatibility.md for more details.
  warnings.warn(


## MMAction2识别模型的推理

In [ ]:
!mkdir -p /data/posec3d/

In [ ]:
wget -P data/posec3d https://download.openmmlab.com/mmaction/posec3d/gym_train.pkl
wget -P data/posec3d https://download.openmmlab.com/mmaction/posec3d/gym_val.pkl

--2023-01-18 21:43:15--  https://download.openmmlab.com/mmaction/posec3d/gym_train.pkl
Resolving download.openmmlab.com (download.openmmlab.com)... 8.48.85.209, 8.48.85.211, 8.48.85.210, ...
Connecting to download.openmmlab.com (download.openmmlab.com)|8.48.85.209|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 129813242 (124M) [application/octet-stream]
Saving to: ‘/content/mmaction2/data/posec3d/gym_train.pkl’

gym_train.pkl       100%[===================>] 123.80M  23.2MB/s    in 5.3s    

2023-01-18 21:43:20 (23.3 MB/s) - ‘/content/mmaction2/data/posec3d/gym_train.pkl’ saved [129813242/129813242]

--2023-01-18 21:43:21--  https://download.openmmlab.com/mmaction/posec3d/gym_val.pkl
Resolving download.openmmlab.com (download.openmmlab.com)... 8.48.85.209, 8.48.85.211, 8.48.85.210, ...
Connecting to download.openmmlab.com (download.openmmlab.com)|8.48.85.209|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 56136430 (54M) [application/

In [ ]:
import pickle
import pandas as pd
f = open('/content/mmaction2/data/posec3d/gym_train.pkl','rb')
list1 = pickle.load(f)

test = pd.DataFrame(data=list1)
test1 = test.loc[:,'frame_dir':'keypoint']
test1.to_csv('test1.csv')


In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
!mv /content/mmaction2/test1.csv /content/drive/MyDrive 

In [ ]:
!python tools/train.py configs/skeleton/posec3d/slowonly_r50_u48_240e_gym_keypoint.py \
    --work-dir work_dirs/slowonly_r50_u48_240e_gym_keypoint \
    --validate --seed 0 --deterministic --gpu-ids 0

/usr/local/lib/python3.8/dist-packages/mmcv/__init__.py:20: UserWarning: On January 1, 2023, MMCV will release v2.0.0, in which it will remove components related to the training process and add a data transformation module. In addition, it will rename the package names mmcv to mmcv-lite and mmcv-full to mmcv. See https://github.com/open-mmlab/mmcv/blob/master/docs/en/compatibility.md for more details.
  warnings.warn(
/content/mmaction2/mmaction/utils/setup_env.py:32: UserWarning: Setting OMP_NUM_THREADS environment variable for each process to be 1 in default, to avoid your system being overloaded, please further tune the variable for optimal performance in your application as needed.
  warnings.warn(
/content/mmaction2/mmaction/utils/setup_env.py:42: UserWarning: Setting MKL_NUM_THREADS environment variable for each process to be 1 in default, to avoid your system being overloaded, please further tune the variable for optimal performance in your application as needed.
  warnings.warn

In [ ]:
!pwd

/content/mmaction2


In [ ]:
# 创建checkpoints文件夹并下载tsn模型
!mkdir checkpoints
!wget -c https://download.openmmlab.com/mmaction/recognition/tsn/tsn_r50_1x1x3_100e_kinetics400_rgb/tsn_r50_1x1x3_100e_kinetics400_rgb_20200614-e508be42.pth \
      -O checkpoints/tsn_r50_1x1x3_100e_kinetics400_rgb_20200614-e508be42.pth

In [ ]:
from mmaction.apis import inference_recognizer, init_recognizer

# 选择tsn对应的配置文件
config = 'configs/recognition/tsn/tsn_r50_video_inference_1x1x3_100e_kinetics400_rgb.py'
# 加载上面下载的checkpoint文件
checkpoint = 'checkpoints/tsn_r50_1x1x3_100e_kinetics400_rgb_20200614-e508be42.pth'
# 初始化模型
model = init_recognizer(config, checkpoint, device='cuda:0')

In [ ]:
# 选择视频进行推理
video = 'demo/demo.mp4'
label = 'tools/data/kinetics/label_map_k400.txt'
results = inference_recognizer(model, video)

labels = open(label).readlines()
labels = [x.strip() for x in labels]
results = [(labels[k[0]], k[1]) for k in results]

In [ ]:
# 查看视频
from IPython.display import HTML
from base64 import b64encode
mp4 = open(video,'rb').read()
data_url = "data:video/mp4;base64," + b64encode(mp4).decode()
HTML("""
<video width=400 controls>
      <source src="%s" type="video/mp4">
</video>
""" % data_url)

In [ ]:
# 查看推理Top-5结果
for result in results:
    print(f'{result[0]}: ', result[1])

## 在自定义数据集上训练模型
训练新模型通常有三个步骤:
- 支持新数据集
- 修改配置文件
- 训练模型


### 支持新数据集

这里我们给出将数据转换为已有数据集格式的示例。其他方法可以参考[doc](/docs/tutorials/new_dataset.md)

用到的是一个从[Kinetics-400](https://deepmind.com/research/open-source/open-source-datasets/kinetics/)中获取的tiny数据集。包含30个训练视频，10个测试视频。

In [ ]:
# 下载并解压数据集kinetics400_tiny
!rm kinetics400_tiny.zip*
!rm -rf kinetics400_tiny
!wget https://download.openmmlab.com/mmaction/kinetics400_tiny.zip
!unzip kinetics400_tiny.zip > /dev/null

rm: cannot remove 'kinetics400_tiny.zip*': No such file or directory
--2022-11-17 04:55:09--  https://download.openmmlab.com/mmaction/kinetics400_tiny.zip
Resolving download.openmmlab.com (download.openmmlab.com)... 47.254.186.218
Connecting to download.openmmlab.com (download.openmmlab.com)|47.254.186.218|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 18308682 (17M) [application/zip]
Saving to: ‘kinetics400_tiny.zip’

kinetics400_tiny.zi 100%[===================>]  17.46M  5.61MB/s    in 3.2s    

2022-11-17 04:55:13 (5.37 MB/s) - ‘kinetics400_tiny.zip’ saved [18308682/18308682]



In [ ]:
# 安装tree工具并检查数据集目录结构
!apt-get -q install tree
!tree kinetics400_tiny

Reading package lists...
Building dependency tree...
Reading state information...
The following package was automatically installed and is no longer required:
  libnvidia-common-460
Use 'apt autoremove' to remove it.
The following NEW packages will be installed:
  tree
0 upgraded, 1 newly installed, 0 to remove and 5 not upgraded.
Need to get 40.7 kB of archives.
After this operation, 105 kB of additional disk space will be used.
Get:1 http://archive.ubuntu.com/ubuntu bionic/universe amd64 tree amd64 1.7.0-5 [40.7 kB]
Fetched 40.7 kB in 0s (717 kB/s)
Selecting previously unselected package tree.
(Reading database ... 123991 files and directories currently installed.)
Preparing to unpack .../tree_1.7.0-5_amd64.deb ...
Unpacking tree (1.7.0-5) ...
Setting up tree (1.7.0-5) ...
Processing triggers for man-db (2.8.3-2ubuntu0.1) ...
kinetics400_tiny
├── kinetics_tiny_train_video.txt
├── kinetics_tiny_val_video.txt
├── train
│   ├── 27_CSXByd3s.mp4
│   ├── 34XczvTaRiI.mp4
│   ├── A-wiliK50Zw

In [ ]:
# 查看标注文件格式
!cat kinetics400_tiny/kinetics_tiny_train_video.txt

D32_1gwq35E.mp4 0
iRuyZSKhHRg.mp4 1
oXy-e_P_cAI.mp4 0
34XczvTaRiI.mp4 1
h2YqqUhnR34.mp4 0
O46YA8tI530.mp4 0
kFC3KY2bOP8.mp4 1
WWP5HZJsg-o.mp4 1
phDqGd0NKoo.mp4 1
yLC9CtWU5ws.mp4 0
27_CSXByd3s.mp4 1
IyfILH9lBRo.mp4 1
T_TMNGzVrDk.mp4 1
TkkZPZHbAKA.mp4 0
PnOe3GZRVX8.mp4 1
soEcZZsBmDs.mp4 1
FMlSTTpN3VY.mp4 1
WaS0qwP46Us.mp4 0
A-wiliK50Zw.mp4 1
oMrZaozOvdQ.mp4 1
ZQV4U2KQ370.mp4 0
DbX8mPslRXg.mp4 1
h10B9SVE-nk.mp4 1
P5M-hAts7MQ.mp4 0
R8HXQkdgKWA.mp4 0
D92m0HsHjcQ.mp4 0
RqnKtCEoEcA.mp4 0
LvcFDgCAXQs.mp4 0
xGY2dP0YUjA.mp4 0
Wh_YPQdH1Zg.mp4 0


根据[`VideoDataset`](./datasets/video_dataset.py)中定义的格式，每一行表示样本视频的文件名和标签，用空格符分隔。


### 修改配置文件

我们需要修改配置文件，同时会用到之前下载的checkpoint作为pre-trained模型。


In [ ]:
# 获得tsn对应的配置文件cfg
from mmcv import Config
cfg = Config.fromfile('./configs/recognition/tsn/tsn_r50_video_1x1x8_100e_kinetics400_rgb.py')

FileNotFoundError: ignored

我们在原本用于kinetics400-full数据集训练的tsn模型配置上进行修改，让模型可以在Kinetics400-tiny数据集上进行训练。



In [ ]:
from mmcv.runner import set_random_seed

# 修改数据集类型和各个文件路径
cfg.dataset_type = 'VideoDataset'
cfg.data_root = 'kinetics400_tiny/train/'
cfg.data_root_val = 'kinetics400_tiny/val/'
cfg.ann_file_train = 'kinetics400_tiny/kinetics_tiny_train_video.txt'
cfg.ann_file_val = 'kinetics400_tiny/kinetics_tiny_val_video.txt'
cfg.ann_file_test = 'kinetics400_tiny/kinetics_tiny_val_video.txt'

cfg.data.test.type = 'VideoDataset'
cfg.data.test.ann_file = 'kinetics400_tiny/kinetics_tiny_val_video.txt'
cfg.data.test.data_prefix = 'kinetics400_tiny/val/'

cfg.data.train.type = 'VideoDataset'
cfg.data.train.ann_file = 'kinetics400_tiny/kinetics_tiny_train_video.txt'
cfg.data.train.data_prefix = 'kinetics400_tiny/train/'

cfg.data.val.type = 'VideoDataset'
cfg.data.val.ann_file = 'kinetics400_tiny/kinetics_tiny_val_video.txt'
cfg.data.val.data_prefix = 'kinetics400_tiny/val/'

# 这里用于确认是否使用到omnisource训练
cfg.setdefault('omnisource', False)
# 修改cls_head中类别数为2
cfg.model.cls_head.num_classes = 2
# 使用预训练好的tsn模型
cfg.load_from = './checkpoints/tsn_r50_1x1x3_100e_kinetics400_rgb_20200614-e508be42.pth'

# 设置工作目录
cfg.work_dir = './tutorial_exps'

# 由于是单卡训练，修改对应的lr
cfg.data.videos_per_gpu = cfg.data.videos_per_gpu // 16
cfg.optimizer.lr = cfg.optimizer.lr / 8 / 16
cfg.total_epochs = 30

# 设置存档点间隔减少存储空间的消耗
cfg.checkpoint_config.interval = 10
# 设置日志打印间隔减少打印时间
cfg.log_config.interval = 5

# 固定随机种子使得结果可复现
cfg.seed = 0
set_random_seed(0, deterministic=False)
cfg.gpu_ids = range(1)

# 打印所有的配置参数
print(f'Config:\n{cfg.pretty_text}')


### 训练识别模型


In [ ]:
import os.path as osp

from mmaction.datasets import build_dataset
from mmaction.models import build_model
from mmaction.apis import train_model

import mmcv

# 构建数据集
datasets = [build_dataset(cfg.data.train)]

# 构建动作识别模型
model = build_model(cfg.model, train_cfg=cfg.get('train_cfg'), test_cfg=cfg.get('test_cfg'))

# 创建工作目录并训练模型
mmcv.mkdir_or_exist(osp.abspath(cfg.work_dir))
train_model(model, datasets, cfg, distributed=False, validate=True)

NameError: ignored

## 评价模型


In [ ]:
from mmaction.apis import single_gpu_test
from mmaction.datasets import build_dataloader
from mmcv.parallel import MMDataParallel

# 构建测试数据集
dataset = build_dataset(cfg.data.test, dict(test_mode=True))
data_loader = build_dataloader(
        dataset,
        videos_per_gpu=1,
        workers_per_gpu=cfg.data.workers_per_gpu,
        dist=False,
        shuffle=False)
model = MMDataParallel(model, device_ids=[0])
outputs = single_gpu_test(model, data_loader)

# 在测试集上评价训练完成的识别模型
eval_config = cfg.evaluation
eval_config.pop('interval')
eval_res = dataset.evaluate(outputs, **eval_config)
for name, val in eval_res.items():
    print(f'{name}: {val:.04f}')

## 时空动作识别

这里我们用到mmdet来辅助完成时空动作识别的任务，首先要在主目录下进行安装。

In [ ]:
# 克隆mmdetection项目
%cd ..
!git clone https://github.com/open-mmlab/mmdetection.git
%cd mmdetection

# 以可编辑的模式安装mmdet
!pip install -e .
%cd ../mmaction2

同时我们需要上传视频至目录mmaction2下

In [ ]:
!wget https://download.openmmlab.com/mmaction/dataset/sample/1j20qq1JyX4.mp4 -O demo/1j20qq1JyX4.mp4

In [ ]:
# 完成时空检测
!python demo/demo_spatiotemporal_det.py --video demo/1j20qq1JyX4.mp4

In [ ]:
# 查看视频
from IPython.display import HTML
from base64 import b64encode
mp4 = open('demo/stdet_demo.mp4','rb').read()
data_url = "data:video/mp4;base64," + b64encode(mp4).decode()
HTML("""
<video width=400 controls>
      <source src="%s" type="video/mp4">
</video>
""" % data_url)